In [1]:
import sgkit as sg
import pandas as pd
import numpy as np
import zarr
import xarray as xr
import allel

In [2]:
import glob

ds = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2L.zarr')
ds

ds2 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2R.zarr')
ds2

ds3 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3L.zarr')
ds3 

ds4 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3R.zarr')
ds4 

ds5 = sg.load_dataset('/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_X.zarr')
ds5 

<xarray.Dataset>
Dimensions:               (variants: 684159, samples: 18, alleles: 4,
                           ploidy: 2, alt_alleles: 3)
Dimensions without coordinates: variants, samples, alleles, ploidy, alt_alleles
Data variables: (12/15)
    call_AD               (variants, samples, alleles) int32 dask.array<chunksize=(10000, 18, 4), meta=np.ndarray>
    call_DP               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_GQ               (variants, samples) int32 dask.array<chunksize=(10000, 18), meta=np.ndarray>
    call_genotype         (variants, samples, ploidy) int8 dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_mask    (variants, samples, ploidy) bool dask.array<chunksize=(10000, 18, 2), meta=np.ndarray>
    call_genotype_phased  (variants, samples) bool dask.array<chunksize=(10000, 18), meta=np.ndarray>
    ...                    ...
    variant_MQ            (variants) float32 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_allele        (variants, alleles) object dask.array<chunksize=(10000, 4), meta=np.ndarray>
    variant_contig        (variants) int8 dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id            (variants) object dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_id_mask       (variants) bool dask.array<chunksize=(10000,), meta=np.ndarray>
    variant_position      (variants) int32 dask.array<chunksize=(10000,), meta=np.ndarray>
Attributes:
    contigs:               ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R...
    max_alt_alleles_seen:  4
    source:                sgkit-0.4.0

In [3]:
def masks_ara_hyb(ds, thres_number):

    ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
    ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
    ds['sample_cohort_name']

    sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
    sample_is_ara = ds['sample_cohort_name'] == 'ara'
    sample_is_gam = ds['sample_cohort_name'] == 'gam'
    
    g = allel.GenotypeArray(ds['call_genotype'])

    is_ara_missing=g[:,sample_is_ara].is_missing().all(axis=1)
    is_ara_hom_ref=g[:,sample_is_ara].is_hom_ref().all(axis=1)
    is_ara_hom_alt=g[:,sample_is_ara].is_hom_alt().all(axis=1)

    is_hyb_hom_alt = g[:,sample_is_hyb].is_hom_alt().sum(axis=1) >= thres_number
    is_hyb_missing=g[:,sample_is_hyb].is_missing().sum(axis=1) >= thres_number
    is_hyb_hom_ref=g[:,sample_is_hyb].is_hom_ref().sum(axis=1) >= thres_number

    mask_ara_no_hyb_alt=is_hyb_hom_alt & is_ara_missing

    mask_ara_ref_hyb_alt=is_hyb_hom_alt & is_ara_hom_ref

    mask_ara_alt_hyb_no=is_hyb_missing & is_ara_hom_alt

    mask_ara_alt_hyb_ref=is_hyb_hom_ref & is_ara_hom_alt

    mask_fin= mask_ara_no_hyb_alt | mask_ara_ref_hyb_alt | mask_ara_alt_hyb_no | mask_ara_alt_hyb_ref

    # return mask_fin.sum()
    return mask_fin

In [5]:
import pyranges as pr

# import annotations
ann_db = pr.read_gff3("/Users/christinagkertsou/Desktop/Finalyearproject/VectorBase-56_AgambiaePEST.gff")
ann_db = ann_db['AgamP4_2L']
ann_db

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_2L,VEuPathDB,protein_coding_gene,38696490,38697681,.,+,.,AGAP006810,chromatin modification-related protein EAF6,NaN,NaN,NaN,NaN,NaN
1,AgamP4_2L,VEuPathDB,mRNA,38696490,38697681,.,+,.,AGAP006810-RA,chromatin modification-related protein EAF6,AGAP006810,NaN,NaN,NaN,NaN
2,AgamP4_2L,VEuPathDB,exon,38696490,38696664,.,+,.,exon_AGAP006810-RA-E1,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
3,AgamP4_2L,VEuPathDB,exon,38696790,38696994,.,+,.,exon_AGAP006810-RA-E2,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
4,AgamP4_2L,VEuPathDB,exon,38697074,38697681,.,+,.,exon_AGAP006810-RA-E3,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43788,AgamP4_2L,VEuPathDB,exon,31566897,31566994,.,-,.,exon_AGAP006408-RA-E1,NaN,AGAP006408-RA,AGAP006408,NaN,NaN,NaN
43789,AgamP4_2L,VEuPathDB,CDS,31565063,31566794,.,-,0,AGAP006408-PA-CDS1,NaN,AGAP006408-RA,AGAP006408,AGAP006408-PA,NaN,NaN
43790,AgamP4_2L,VEuPathDB,three_prime_UTR,31564827,31565063,.,-,.,utr_AGAP006408-RA_1,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN
43791,AgamP4_2L,VEuPathDB,five_prime_UTR,31566794,31566825,.,-,.,utr_AGAP006408-RA_2,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN


In [6]:
num_snps_new= {}

for contig in ['AgamP4_2L', 'AgamP4_2R', 'AgamP4_3L', 'AgamP4_3R', 'AgamP4_X']:
    print(contig)
    ds = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/{contig}.zarr')
    num_snps_new[contig] = []
   

    for threshold in [2, 3, 4, 5]:
        print(threshold,)
        n = masks_ara_hyb(ds, threshold)
        num_snps_new[contig].append(n)
        print(n)

AgamP4_2L
2
[False False False ...  True  True  True]
3
[False False False ...  True  True  True]
4
[False False False ...  True  True  True]
5
[False False False ...  True  True  True]
AgamP4_2R
2
[False  True False ... False False False]
3
[False  True False ... False False False]
4
[False  True False ... False False False]
5
[False  True False ... False False False]
AgamP4_3L
2
[False False False ...  True  True False]
3
[False False False ...  True  True False]
4
[False False False ...  True  True False]
5
[False False False ...  True  True False]
AgamP4_3R
2
[ True  True False ... False False False]
3
[ True  True False ... False False False]
4
[ True  True False ... False False False]
5
[ True  True False ... False False False]
AgamP4_X
2
[ True False  True ... False False False]
3
[ True False  True ... False False False]
4
[ True False  True ... False False False]
5
[ True False  True ... False False False]


In [ ]:
#2L starts here

In [7]:
ds = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2L.zarr')

ds['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds['sample_cohort_name']

sample_is_hyb = ds['sample_cohort_name'] == 'hyb'
sample_is_ara = ds['sample_cohort_name'] == 'ara'
sample_is_gam = ds['sample_cohort_name'] == 'gam'

g = allel.GenotypeArray(ds['call_genotype'])

In [8]:
# snp mask
g_is_called_ara_SNP=g[:,sample_is_ara].is_called().all(axis=1)
g_is_called_hyb_SNP=g[:,sample_is_hyb].is_called().sum(axis=1) >=3

mask1=g_is_called_ara_SNP | g_is_called_hyb_SNP

In [9]:
# het mask
is_ara_alt_hom_SNP=g[:,sample_is_ara].is_hom_alt().all(axis=1)
is_hyb_het_SNP=g[:,sample_is_hyb].is_het().sum(axis=1)>=3
is_ara_hom_ref_SNP=g[:,sample_is_ara].is_hom_ref().all(axis=1)

mask2=(is_ara_alt_hom_SNP | is_ara_hom_ref_SNP) & is_hyb_het_SNP

In [10]:
pos = allel.SortedIndex(ds['variant_position'])

pos_called_all_SNPs = pos[mask1]
pos_called_hets = pos[mask2]


In [11]:
import pyranges as pr

pr_all_snps = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_called_all_SNPs])
pr_hets = pr.from_dict([{'Chromosome': 'AgamP4_2L', 'Start': p, 'End': p} for p in pos_called_hets])

count_SNP_df = pr.count_overlaps({'SNPs': pr_all_snps, 'selected_hets': pr_hets}, features=ann_db).df
count_SNP_df

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_2L,VEuPathDB,protein_coding_gene,38696490,38697681,.,+,.,AGAP006810,chromatin modification-related protein EAF6,NaN,NaN,NaN,NaN,NaN,25,0
1,AgamP4_2L,VEuPathDB,mRNA,38696490,38697681,.,+,.,AGAP006810-RA,chromatin modification-related protein EAF6,AGAP006810,NaN,NaN,NaN,NaN,25,0
2,AgamP4_2L,VEuPathDB,exon,38696490,38696664,.,+,.,exon_AGAP006810-RA-E1,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,5,0
3,AgamP4_2L,VEuPathDB,exon,38696790,38696994,.,+,.,exon_AGAP006810-RA-E2,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,6,0
4,AgamP4_2L,VEuPathDB,exon,38697074,38697681,.,+,.,exon_AGAP006810-RA-E3,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43788,AgamP4_2L,VEuPathDB,exon,31566897,31566994,.,-,.,exon_AGAP006408-RA-E1,NaN,AGAP006408-RA,AGAP006408,NaN,NaN,NaN,2,0
43789,AgamP4_2L,VEuPathDB,CDS,31565063,31566794,.,-,0,AGAP006408-PA-CDS1,NaN,AGAP006408-RA,AGAP006408,AGAP006408-PA,NaN,NaN,15,0
43790,AgamP4_2L,VEuPathDB,three_prime_UTR,31564827,31565063,.,-,.,utr_AGAP006408-RA_1,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN,7,0
43791,AgamP4_2L,VEuPathDB,five_prime_UTR,31566794,31566825,.,-,.,utr_AGAP006408-RA_2,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN,1,0


In [12]:
count_SNP_df['selected_ratio'] = count_SNP_df['selected_hets'] / count_SNP_df['SNPs']
count_SNP_df

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_2L,VEuPathDB,protein_coding_gene,38696490,38697681,.,+,.,AGAP006810,chromatin modification-related protein EAF6,NaN,NaN,NaN,NaN,NaN,25,0,0.0
1,AgamP4_2L,VEuPathDB,mRNA,38696490,38697681,.,+,.,AGAP006810-RA,chromatin modification-related protein EAF6,AGAP006810,NaN,NaN,NaN,NaN,25,0,0.0
2,AgamP4_2L,VEuPathDB,exon,38696490,38696664,.,+,.,exon_AGAP006810-RA-E1,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,5,0,0.0
3,AgamP4_2L,VEuPathDB,exon,38696790,38696994,.,+,.,exon_AGAP006810-RA-E2,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,6,0,0.0
4,AgamP4_2L,VEuPathDB,exon,38697074,38697681,.,+,.,exon_AGAP006810-RA-E3,NaN,AGAP006810-RA,AGAP006810,NaN,NaN,NaN,5,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43788,AgamP4_2L,VEuPathDB,exon,31566897,31566994,.,-,.,exon_AGAP006408-RA-E1,NaN,AGAP006408-RA,AGAP006408,NaN,NaN,NaN,2,0,0.0
43789,AgamP4_2L,VEuPathDB,CDS,31565063,31566794,.,-,0,AGAP006408-PA-CDS1,NaN,AGAP006408-RA,AGAP006408,AGAP006408-PA,NaN,NaN,15,0,0.0
43790,AgamP4_2L,VEuPathDB,three_prime_UTR,31564827,31565063,.,-,.,utr_AGAP006408-RA_1,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN,7,0,0.0
43791,AgamP4_2L,VEuPathDB,five_prime_UTR,31566794,31566825,.,-,.,utr_AGAP006408-RA_2,NaN,AGAP006408-RA,NaN,NaN,NaN,NaN,1,0,0.0


In [12]:
count_SNP_df.to_csv("Snps_hets.csv")

In [60]:
grouped_2L = count_SNP_df.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2L = grouped_2L[['gene_id', 'SNPs', 'selected_hets']]
grouped_2L['ratio'] = grouped_2L['selected_hets'] / grouped_2L['SNPs']
grouped_2L.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
2295,AGAP007170,17,13,0.764706
829,AGAP005581,5,3,0.600000
163,AGAP004853,2,1,0.500000
680,AGAP005422,9,4,0.444444
824,AGAP005576,17,6,0.352941
...,...,...,...,...
2919,AGAP029027,0,0,NaN
3051,AGAP029798,0,0,NaN
3076,AGAP029842,0,0,NaN
3083,AGAP029849,0,0,NaN


In [65]:
grouped_2L.to_csv("Genes_2L_spreasheet.csv")

In [ ]:
#2R starts here

In [13]:
import pyranges as pr

# import annotations
ann_db_2R = pr.read_gff3("/Users/christinagkertsou/Desktop/VectorBase-56_AgambiaePEST.gff")
ann_db_2R = ann_db_2R['AgamP4_2R']
ann_db_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN


In [14]:
ds_2R = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_2R.zarr')

ds_2R['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds_2R['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds_2R['sample_cohort_name']

sample_is_hyb_2R = ds_2R['sample_cohort_name'] == 'hyb'
sample_is_ara_2R = ds_2R['sample_cohort_name'] == 'ara'
sample_is_gam_2R = ds_2R['sample_cohort_name'] == 'gam'

g_2R = allel.GenotypeArray(ds_2R['call_genotype'])

In [15]:
# snp mask
g_is_called_ara_SNP_2R=g_2R[:,sample_is_ara].is_called().all(axis=1)
g_is_called_hyb_SNP_2R=g_2R[:,sample_is_hyb].is_called().sum(axis=1) >=3

mask1_2R=g_is_called_ara_SNP_2R | g_is_called_hyb_SNP_2R

In [16]:
# het mask
is_ara_alt_hom_SNP_2R=g_2R[:,sample_is_ara].is_hom_alt().all(axis=1)
is_hyb_het_SNP_2R=g_2R[:,sample_is_hyb].is_het().sum(axis=1)>=3

mask2_2R=is_ara_alt_hom_SNP_2R & is_hyb_het_SNP_2R

In [19]:
pos_2R = allel.SortedIndex(ds_2R['variant_position'])

pos_called_all_SNPs_2R = pos_2R[mask1_2R]
pos_called_hets_2R = pos_2R[mask2_2R]

In [20]:
import pyranges as pr

pr_all_snps_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_called_all_SNPs_2R])
pr_hets_2R = pr.from_dict([{'Chromosome': 'AgamP4_2R', 'Start': p, 'End': p} for p in pos_called_hets_2R])

count_SNP_df_2R = pr.count_overlaps({'SNPs': pr_all_snps_2R, 'selected_hets': pr_hets_2R}, features=ann_db_2R).df
count_SNP_df_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN,167,17
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN,167,17
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,1,0
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,4,1
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,58,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN,38,0
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,32,0
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN,1,0


In [21]:
count_SNP_df_2R['selected_ratio'] = count_SNP_df_2R['selected_hets'] / count_SNP_df_2R['SNPs']
count_SNP_df_2R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_2R,VEuPathDB,protein_coding_gene,24245442,24251444,.,+,.,AGAP002619,unspecified product,NaN,NaN,NaN,NaN,NaN,167,17,0.101796
1,AgamP4_2R,VEuPathDB,mRNA,24245442,24251444,.,+,.,AGAP002619-RA,unspecified product,AGAP002619,NaN,NaN,NaN,NaN,167,17,0.101796
2,AgamP4_2R,VEuPathDB,exon,24245442,24245526,.,+,.,exon_AGAP002619-RA-E1,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,1,0,0.000000
3,AgamP4_2R,VEuPathDB,exon,24245644,24246259,.,+,.,exon_AGAP002619-RA-E2,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,4,1,0.250000
4,AgamP4_2R,VEuPathDB,exon,24247851,24249256,.,+,.,exon_AGAP002619-RA-E3,NaN,AGAP002619-RA,AGAP002619,NaN,NaN,NaN,58,5,0.086207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53185,AgamP4_2R,VEuPathDB,exon,47020993,47023594,.,-,.,exon_AGAP029680-RA-E1,NaN,AGAP029680-RA,AGAP029680,NaN,NaN,NaN,38,0,0.000000
53186,AgamP4_2R,VEuPathDB,CDS,47018304,47020467,.,-,0,AGAP029680-PA-CDS2,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,32,0,0.000000
53187,AgamP4_2R,VEuPathDB,CDS,47020993,47021209,.,-,0,AGAP029680-PA-CDS1,NaN,AGAP029680-RA,AGAP029680,AGAP029680-PA,NaN,NaN,0,0,NaN
53188,AgamP4_2R,VEuPathDB,three_prime_UTR,47017982,47018304,.,-,.,utr_AGAP029680-RA_1,NaN,AGAP029680-RA,NaN,NaN,NaN,NaN,1,0,0.000000


In [59]:
grouped_2R = count_SNP_df_2R.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_2R = grouped_2R[['gene_id', 'SNPs', 'selected_hets']]
grouped_2R['ratio'] = grouped_2R['selected_hets'] / grouped_2R['SNPs']
grouped_2R.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
1453,AGAP002705,34,31,0.911765
1059,AGAP002283,25,21,0.840000
1432,AGAP002680,53,40,0.754717
1063,AGAP002287,8,6,0.750000
1434,AGAP002684,32,24,0.750000
...,...,...,...,...
3677,AGAP028769,0,0,NaN
3679,AGAP028771,0,0,NaN
3694,AGAP029088,0,0,NaN
3783,AGAP029683,0,0,NaN


In [66]:
grouped_2R.to_csv("Genes_2R_spreasheet.csv")

In [22]:
count_SNP_df_2R.to_csv("Snps_hets_2R.csv")

In [ ]:
#3R starts here

In [23]:
import pyranges as pr

# import annotations
ann_db_3R = pr.read_gff3("/Users/christinagkertsou/Desktop/VectorBase-56_AgambiaePEST.gff")
ann_db_3R = ann_db_3R['AgamP4_3R']
ann_db_3R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_3R,VEuPathDB,protein_coding_gene,43313157,43318009,.,+,.,AGAP009791,neprilysin%2C neutral endopeptidase 1,NaN,NaN,NaN,Nep1,NaN
1,AgamP4_3R,VEuPathDB,mRNA,43313157,43318009,.,+,.,AGAP009791-RA,neprilysin%2C neutral endopeptidase 1,AGAP009791,NaN,NaN,NaN,NaN
2,AgamP4_3R,VEuPathDB,exon,43313157,43314541,.,+,.,exon_AGAP009791-RA-E1,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN
3,AgamP4_3R,VEuPathDB,exon,43315148,43315276,.,+,.,exon_AGAP009791-RA-E2,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN
4,AgamP4_3R,VEuPathDB,exon,43315443,43315561,.,+,.,exon_AGAP009791-RA-E3,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,AgamP4_3R,VEuPathDB,CDS,44595892,44596238,.,-,1,AGAP009868-PA-CDS3,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN
39608,AgamP4_3R,VEuPathDB,CDS,44596356,44596499,.,-,0,AGAP009868-PA-CDS2,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN
39609,AgamP4_3R,VEuPathDB,CDS,44596716,44596728,.,-,0,AGAP009868-PA-CDS1,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN
39610,AgamP4_3R,VEuPathDB,three_prime_UTR,44595743,44595892,.,-,.,utr_AGAP009868-RA_1,NaN,AGAP009868-RA,NaN,NaN,NaN,NaN


In [24]:
ds_3R = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3R.zarr')

ds_3R['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds_3R['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds_3R['sample_cohort_name']

sample_is_hyb_3R = ds_3R['sample_cohort_name'] == 'hyb'
sample_is_ara_3R = ds_3R['sample_cohort_name'] == 'ara'
sample_is_gam_3R = ds_3R['sample_cohort_name'] == 'gam'

g_3R = allel.GenotypeArray(ds_3R['call_genotype'])

In [25]:
# snp mask
g_is_called_ara_SNP_3R=g_3R[:,sample_is_ara].is_called().all(axis=1)
g_is_called_hyb_SNP_3R=g_3R[:,sample_is_hyb].is_called().sum(axis=1) >=3

mask1_3R=g_is_called_ara_SNP_3R | g_is_called_hyb_SNP_3R

In [26]:
# het mask
is_ara_alt_hom_SNP_3R=g_3R[:,sample_is_ara].is_hom_alt().all(axis=1)
is_hyb_het_SNP_3R=g_3R[:,sample_is_hyb].is_het().sum(axis=1)>=3

mask2_3R=is_ara_alt_hom_SNP_3R & is_hyb_het_SNP_3R

In [27]:
pos_3R = allel.SortedIndex(ds_3R['variant_position'])

pos_called_all_SNPs_3R = pos_3R[mask1_3R]
pos_called_hets_3R = pos_3R[mask2_3R]

In [28]:
import pyranges as pr

pr_all_snps_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_called_all_SNPs_3R])
pr_hets_3R = pr.from_dict([{'Chromosome': 'AgamP4_3R', 'Start': p, 'End': p} for p in pos_called_hets_3R])

count_SNP_df_3R = pr.count_overlaps({'SNPs': pr_all_snps_3R, 'selected_hets': pr_hets_3R}, features=ann_db_3R).df
count_SNP_df_3R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_3R,VEuPathDB,protein_coding_gene,43313157,43318009,.,+,.,AGAP009791,neprilysin%2C neutral endopeptidase 1,NaN,NaN,NaN,Nep1,NaN,93,0
1,AgamP4_3R,VEuPathDB,mRNA,43313157,43318009,.,+,.,AGAP009791-RA,neprilysin%2C neutral endopeptidase 1,AGAP009791,NaN,NaN,NaN,NaN,93,0
2,AgamP4_3R,VEuPathDB,exon,43313157,43314541,.,+,.,exon_AGAP009791-RA-E1,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,32,0
3,AgamP4_3R,VEuPathDB,exon,43315148,43315276,.,+,.,exon_AGAP009791-RA-E2,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,0,0
4,AgamP4_3R,VEuPathDB,exon,43315443,43315561,.,+,.,exon_AGAP009791-RA-E3,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,AgamP4_3R,VEuPathDB,CDS,44595892,44596238,.,-,1,AGAP009868-PA-CDS3,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,2,0
39608,AgamP4_3R,VEuPathDB,CDS,44596356,44596499,.,-,0,AGAP009868-PA-CDS2,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,2,0
39609,AgamP4_3R,VEuPathDB,CDS,44596716,44596728,.,-,0,AGAP009868-PA-CDS1,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,0,0
39610,AgamP4_3R,VEuPathDB,three_prime_UTR,44595743,44595892,.,-,.,utr_AGAP009868-RA_1,NaN,AGAP009868-RA,NaN,NaN,NaN,NaN,3,0


In [29]:
count_SNP_df_3R['selected_ratio'] = count_SNP_df_3R['selected_hets'] / count_SNP_df_3R['SNPs']
count_SNP_df_3R

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_3R,VEuPathDB,protein_coding_gene,43313157,43318009,.,+,.,AGAP009791,neprilysin%2C neutral endopeptidase 1,NaN,NaN,NaN,Nep1,NaN,93,0,0.0
1,AgamP4_3R,VEuPathDB,mRNA,43313157,43318009,.,+,.,AGAP009791-RA,neprilysin%2C neutral endopeptidase 1,AGAP009791,NaN,NaN,NaN,NaN,93,0,0.0
2,AgamP4_3R,VEuPathDB,exon,43313157,43314541,.,+,.,exon_AGAP009791-RA-E1,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,32,0,0.0
3,AgamP4_3R,VEuPathDB,exon,43315148,43315276,.,+,.,exon_AGAP009791-RA-E2,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,0,0,NaN
4,AgamP4_3R,VEuPathDB,exon,43315443,43315561,.,+,.,exon_AGAP009791-RA-E3,NaN,AGAP009791-RA,AGAP009791,NaN,NaN,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39607,AgamP4_3R,VEuPathDB,CDS,44595892,44596238,.,-,1,AGAP009868-PA-CDS3,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,2,0,0.0
39608,AgamP4_3R,VEuPathDB,CDS,44596356,44596499,.,-,0,AGAP009868-PA-CDS2,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,2,0,0.0
39609,AgamP4_3R,VEuPathDB,CDS,44596716,44596728,.,-,0,AGAP009868-PA-CDS1,NaN,AGAP009868-RA,AGAP009868,AGAP009868-PA,NaN,NaN,0,0,NaN
39610,AgamP4_3R,VEuPathDB,three_prime_UTR,44595743,44595892,.,-,.,utr_AGAP009868-RA_1,NaN,AGAP009868-RA,NaN,NaN,NaN,NaN,3,0,0.0


In [30]:
count_SNP_df_3R.to_csv("Snps_hets_3R.csv")

In [61]:
grouped_3R = count_SNP_df_3R.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3R = grouped_3R[['gene_id', 'SNPs', 'selected_hets']]
grouped_3R['ratio'] = grouped_3R['selected_hets'] / grouped_3R['SNPs']
grouped_3R.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
887,AGAP008729,16,4,0.250000
878,AGAP008719,16,3,0.187500
900,AGAP008746,63,8,0.126984
902,AGAP008748,68,7,0.102941
901,AGAP008747,93,7,0.075269
...,...,...,...,...
2656,AGAP029419,0,0,NaN
2660,AGAP029423,0,0,NaN
2664,AGAP029427,0,0,NaN
2732,AGAP029712,0,0,NaN


In [67]:
grouped_3R.to_csv("Genes_3R_spreasheet.csv")

In [ ]:
#3L starts here

In [31]:
import pyranges as pr

# import annotations
ann_db_3L = pr.read_gff3("/Users/christinagkertsou/Desktop/VectorBase-56_AgambiaePEST.gff")
ann_db_3L = ann_db_3L['AgamP4_3L']
ann_db_3L

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_3L,VEuPathDB,ncRNA_gene,20375981,20376054,.,+,.,AGAP011265,unspecified product,NaN,NaN,NaN,tRNA-Val,NaN
1,AgamP4_3L,VEuPathDB,tRNA,20375981,20376054,.,+,.,AGAP011265-RA,unspecified product,AGAP011265,NaN,NaN,NaN,NaN
2,AgamP4_3L,VEuPathDB,exon,20375981,20376054,.,+,.,exon_AGAP011265-RA-E1,NaN,AGAP011265-RA,AGAP011265,NaN,NaN,NaN
3,AgamP4_3L,VEuPathDB,protein_coding_gene,19483803,19485190,.,+,.,AGAP011220,Borealin,NaN,NaN,NaN,NaN,NaN
4,AgamP4_3L,VEuPathDB,mRNA,19483803,19485190,.,+,.,AGAP011220-RA,Borealin,AGAP011220,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30214,AgamP4_3L,VEuPathDB,exon,37999395,38000240,.,-,.,exon_AGAP012135-RA-E2,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN
30215,AgamP4_3L,VEuPathDB,exon,38000315,38000592,.,-,.,exon_AGAP012135-RA-E1,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN
30216,AgamP4_3L,VEuPathDB,CDS,37999395,38000240,.,-,2,AGAP012135-PA-CDS2,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN
30217,AgamP4_3L,VEuPathDB,CDS,38000315,38000487,.,-,0,AGAP012135-PA-CDS1,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN


In [32]:
ds_3L = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_3L.zarr')

ds_3L['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds_3L['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds_3L['sample_cohort_name']

sample_is_hyb_3L = ds_3L['sample_cohort_name'] == 'hyb'
sample_is_ara_3L = ds_3L['sample_cohort_name'] == 'ara'
sample_is_gam_3L = ds_3L['sample_cohort_name'] == 'gam'

g_3L = allel.GenotypeArray(ds_3L['call_genotype'])

In [36]:
# snp mask
g_is_called_ara_SNP_3L=g_3L[:,sample_is_ara].is_called().all(axis=1)
g_is_called_hyb_SNP_3L=g_3L[:,sample_is_hyb].is_called().sum(axis=1) >=3

mask1_3L=g_is_called_ara_SNP_3L | g_is_called_hyb_SNP_3L

In [37]:
# het mask
is_ara_alt_hom_SNP_3L=g_3L[:,sample_is_ara].is_hom_alt().all(axis=1)
is_hyb_het_SNP_3L=g_3L[:,sample_is_hyb].is_het().sum(axis=1)>=3

mask2_3L=is_ara_alt_hom_SNP_3L & is_hyb_het_SNP_3L

In [38]:
pos_3L = allel.SortedIndex(ds_3L['variant_position'])

pos_called_all_SNPs_3L = pos_3L[mask1_3L]
pos_called_hets_3L = pos_3L[mask2_3L]

In [39]:
import pyranges as pr

pr_all_snps_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_called_all_SNPs_3L])
pr_hets_3L = pr.from_dict([{'Chromosome': 'AgamP4_3L', 'Start': p, 'End': p} for p in pos_called_hets_3L])

count_SNP_df_3L = pr.count_overlaps({'SNPs': pr_all_snps_3L, 'selected_hets': pr_hets_3L}, features=ann_db_3L).df
count_SNP_df_3L

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_3L,VEuPathDB,ncRNA_gene,20375981,20376054,.,+,.,AGAP011265,unspecified product,NaN,NaN,NaN,tRNA-Val,NaN,0,0
1,AgamP4_3L,VEuPathDB,tRNA,20375981,20376054,.,+,.,AGAP011265-RA,unspecified product,AGAP011265,NaN,NaN,NaN,NaN,0,0
2,AgamP4_3L,VEuPathDB,exon,20375981,20376054,.,+,.,exon_AGAP011265-RA-E1,NaN,AGAP011265-RA,AGAP011265,NaN,NaN,NaN,0,0
3,AgamP4_3L,VEuPathDB,protein_coding_gene,19483803,19485190,.,+,.,AGAP011220,Borealin,NaN,NaN,NaN,NaN,NaN,41,0
4,AgamP4_3L,VEuPathDB,mRNA,19483803,19485190,.,+,.,AGAP011220-RA,Borealin,AGAP011220,NaN,NaN,NaN,NaN,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30214,AgamP4_3L,VEuPathDB,exon,37999395,38000240,.,-,.,exon_AGAP012135-RA-E2,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN,13,0
30215,AgamP4_3L,VEuPathDB,exon,38000315,38000592,.,-,.,exon_AGAP012135-RA-E1,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN,5,0
30216,AgamP4_3L,VEuPathDB,CDS,37999395,38000240,.,-,2,AGAP012135-PA-CDS2,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN,13,0
30217,AgamP4_3L,VEuPathDB,CDS,38000315,38000487,.,-,0,AGAP012135-PA-CDS1,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN,1,0


In [40]:
count_SNP_df_3L['selected_ratio'] = count_SNP_df_3L['selected_hets'] / count_SNP_df_3L['SNPs']
count_SNP_df_3L

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_3L,VEuPathDB,ncRNA_gene,20375981,20376054,.,+,.,AGAP011265,unspecified product,NaN,NaN,NaN,tRNA-Val,NaN,0,0,NaN
1,AgamP4_3L,VEuPathDB,tRNA,20375981,20376054,.,+,.,AGAP011265-RA,unspecified product,AGAP011265,NaN,NaN,NaN,NaN,0,0,NaN
2,AgamP4_3L,VEuPathDB,exon,20375981,20376054,.,+,.,exon_AGAP011265-RA-E1,NaN,AGAP011265-RA,AGAP011265,NaN,NaN,NaN,0,0,NaN
3,AgamP4_3L,VEuPathDB,protein_coding_gene,19483803,19485190,.,+,.,AGAP011220,Borealin,NaN,NaN,NaN,NaN,NaN,41,0,0.0
4,AgamP4_3L,VEuPathDB,mRNA,19483803,19485190,.,+,.,AGAP011220-RA,Borealin,AGAP011220,NaN,NaN,NaN,NaN,41,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30214,AgamP4_3L,VEuPathDB,exon,37999395,38000240,.,-,.,exon_AGAP012135-RA-E2,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN,13,0,0.0
30215,AgamP4_3L,VEuPathDB,exon,38000315,38000592,.,-,.,exon_AGAP012135-RA-E1,NaN,AGAP012135-RA,AGAP012135,NaN,NaN,NaN,5,0,0.0
30216,AgamP4_3L,VEuPathDB,CDS,37999395,38000240,.,-,2,AGAP012135-PA-CDS2,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN,13,0,0.0
30217,AgamP4_3L,VEuPathDB,CDS,38000315,38000487,.,-,0,AGAP012135-PA-CDS1,NaN,AGAP012135-RA,AGAP012135,AGAP012135-PA,NaN,NaN,1,0,0.0


In [41]:
count_SNP_df_3L.to_csv("Snps_hets_3L.csv")

In [62]:
grouped_3L = count_SNP_df_3L.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_3L = grouped_3L[['gene_id', 'SNPs', 'selected_hets']]
grouped_3L['ratio'] = grouped_3L['selected_hets'] / grouped_3L['SNPs']
grouped_3L.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
1624,AGAP012267,9,3,0.333333
1648,AGAP012301,35,9,0.257143
1636,AGAP012283,33,7,0.212121
1628,AGAP012272,19,4,0.210526
1653,AGAP012306,20,4,0.200000
...,...,...,...,...
2099,AGAP029312,0,0,NaN
2211,AGAP029714,0,0,NaN
2222,AGAP029725,0,0,NaN
2229,AGAP029732,0,0,NaN


In [68]:
grouped_3L.to_csv("Genes_3L_spreasheet.csv")

In [ ]:
#X starts here

In [42]:
import pyranges as pr

# import annotations
ann_db_X = pr.read_gff3("/Users/christinagkertsou/Desktop/VectorBase-56_AgambiaePEST.gff")
ann_db_X = ann_db_X['AgamP4_X']
ann_db_X

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note
0,AgamP4_X,VEuPathDB,protein_coding_gene,4606869,4609898,.,+,.,AGAP000252,oligopeptidase A,NaN,NaN,NaN,NaN,NaN
1,AgamP4_X,VEuPathDB,mRNA,4606869,4609898,.,+,.,AGAP000252-RA,oligopeptidase A,AGAP000252,NaN,NaN,NaN,NaN
2,AgamP4_X,VEuPathDB,exon,4606869,4607242,.,+,.,exon_AGAP000252-RA-E1,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN
3,AgamP4_X,VEuPathDB,exon,4607395,4609898,.,+,.,exon_AGAP000252-RA-E2,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN
4,AgamP4_X,VEuPathDB,CDS,4607156,4607242,.,+,0,AGAP000252-PA-CDS1,NaN,AGAP000252-RA,AGAP000252,AGAP000252-PA,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16833,AgamP4_X,VEuPathDB,exon,11258097,11258406,.,-,.,exon_AGAP000623-RA-E1,NaN,AGAP000623-RA,AGAP000623,NaN,NaN,NaN
16834,AgamP4_X,VEuPathDB,CDS,11255397,11258013,.,-,0,AGAP000623-PA-CDS2,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN
16835,AgamP4_X,VEuPathDB,CDS,11258097,11258226,.,-,0,AGAP000623-PA-CDS1,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN
16836,AgamP4_X,VEuPathDB,three_prime_UTR,11254492,11255397,.,-,.,utr_AGAP000623-RA_1,NaN,AGAP000623-RA,NaN,NaN,NaN,NaN


In [43]:
ds_X = sg.load_dataset(f'/Users/christinagkertsou/Desktop/Desktop March/FYP_new_x/dataset/AgamP4_X.zarr')

ds_X['sample_cohort'] = xr.DataArray([0] * 2 + [1] * 10 + [2] * 6, dims="samples")
ds_X['sample_cohort_name'] = xr.DataArray(['ara'] * 2 + ['gam'] * 10 + ['hyb_ctr'] + ['hyb'] * 5, dims="samples")
ds_X['sample_cohort_name']

sample_is_hyb_X = ds_X['sample_cohort_name'] == 'hyb'
sample_is_ara_X = ds_X['sample_cohort_name'] == 'ara'
sample_is_gam_X = ds_X['sample_cohort_name'] == 'gam'

g_X = allel.GenotypeArray(ds_X['call_genotype'])

In [44]:
# snp mask
g_is_called_ara_SNP_X=g_X[:,sample_is_ara].is_called().all(axis=1)
g_is_called_hyb_SNP_X=g_X[:,sample_is_hyb].is_called().sum(axis=1) >=3

mask1_X=g_is_called_ara_SNP_X | g_is_called_hyb_SNP_X

In [45]:
# het mask
is_ara_alt_hom_SNP_X=g_X[:,sample_is_ara].is_hom_alt().all(axis=1)
is_hyb_het_SNP_X=g_X[:,sample_is_hyb].is_het().sum(axis=1)>=3

mask2_X=is_ara_alt_hom_SNP_X & is_hyb_het_SNP_X

In [46]:
pos_X = allel.SortedIndex(ds_X['variant_position'])

pos_called_all_SNPs_X = pos_X[mask1_X]
pos_called_hets_X = pos_X[mask2_X]

In [47]:
import pyranges as pr

pr_all_snps_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_called_all_SNPs_X])
pr_hets_X = pr.from_dict([{'Chromosome': 'AgamP4_X', 'Start': p, 'End': p} for p in pos_called_hets_X])

count_SNP_df_X = pr.count_overlaps({'SNPs': pr_all_snps_X, 'selected_hets': pr_hets_X}, features=ann_db_X).df
count_SNP_df_X

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets
0,AgamP4_X,VEuPathDB,protein_coding_gene,4606869,4609898,.,+,.,AGAP000252,oligopeptidase A,NaN,NaN,NaN,NaN,NaN,58,0
1,AgamP4_X,VEuPathDB,mRNA,4606869,4609898,.,+,.,AGAP000252-RA,oligopeptidase A,AGAP000252,NaN,NaN,NaN,NaN,58,0
2,AgamP4_X,VEuPathDB,exon,4606869,4607242,.,+,.,exon_AGAP000252-RA-E1,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN,10,0
3,AgamP4_X,VEuPathDB,exon,4607395,4609898,.,+,.,exon_AGAP000252-RA-E2,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN,41,0
4,AgamP4_X,VEuPathDB,CDS,4607156,4607242,.,+,0,AGAP000252-PA-CDS1,NaN,AGAP000252-RA,AGAP000252,AGAP000252-PA,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16833,AgamP4_X,VEuPathDB,exon,11258097,11258406,.,-,.,exon_AGAP000623-RA-E1,NaN,AGAP000623-RA,AGAP000623,NaN,NaN,NaN,14,0
16834,AgamP4_X,VEuPathDB,CDS,11255397,11258013,.,-,0,AGAP000623-PA-CDS2,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN,70,0
16835,AgamP4_X,VEuPathDB,CDS,11258097,11258226,.,-,0,AGAP000623-PA-CDS1,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN,2,0
16836,AgamP4_X,VEuPathDB,three_prime_UTR,11254492,11255397,.,-,.,utr_AGAP000623-RA_1,NaN,AGAP000623-RA,NaN,NaN,NaN,NaN,25,0


In [48]:
count_SNP_df_X['selected_ratio'] = count_SNP_df_X['selected_hets'] / count_SNP_df_X['SNPs']
count_SNP_df_X

,Chromosome,Source,Feature,Start,End,Score,Strand,Frame,ID,description,Parent,gene_id,protein_source_id,Name,Note,SNPs,selected_hets,selected_ratio
0,AgamP4_X,VEuPathDB,protein_coding_gene,4606869,4609898,.,+,.,AGAP000252,oligopeptidase A,NaN,NaN,NaN,NaN,NaN,58,0,0.0
1,AgamP4_X,VEuPathDB,mRNA,4606869,4609898,.,+,.,AGAP000252-RA,oligopeptidase A,AGAP000252,NaN,NaN,NaN,NaN,58,0,0.0
2,AgamP4_X,VEuPathDB,exon,4606869,4607242,.,+,.,exon_AGAP000252-RA-E1,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN,10,0,0.0
3,AgamP4_X,VEuPathDB,exon,4607395,4609898,.,+,.,exon_AGAP000252-RA-E2,NaN,AGAP000252-RA,AGAP000252,NaN,NaN,NaN,41,0,0.0
4,AgamP4_X,VEuPathDB,CDS,4607156,4607242,.,+,0,AGAP000252-PA-CDS1,NaN,AGAP000252-RA,AGAP000252,AGAP000252-PA,NaN,NaN,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16833,AgamP4_X,VEuPathDB,exon,11258097,11258406,.,-,.,exon_AGAP000623-RA-E1,NaN,AGAP000623-RA,AGAP000623,NaN,NaN,NaN,14,0,0.0
16834,AgamP4_X,VEuPathDB,CDS,11255397,11258013,.,-,0,AGAP000623-PA-CDS2,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN,70,0,0.0
16835,AgamP4_X,VEuPathDB,CDS,11258097,11258226,.,-,0,AGAP000623-PA-CDS1,NaN,AGAP000623-RA,AGAP000623,AGAP000623-PA,NaN,NaN,2,0,0.0
16836,AgamP4_X,VEuPathDB,three_prime_UTR,11254492,11255397,.,-,.,utr_AGAP000623-RA_1,NaN,AGAP000623-RA,NaN,NaN,NaN,NaN,25,0,0.0


In [49]:
count_SNP_df_X.to_csv("Snps_hets_X.csv")

In [63]:
grouped_X = count_SNP_df_X.query("Feature == 'exon'").groupby('gene_id').aggregate(sum).reset_index()
grouped_X = grouped_X[['gene_id', 'SNPs', 'selected_hets']]
grouped_X['ratio'] = grouped_X['selected_hets'] / grouped_X['SNPs']
grouped_X.sort_values('ratio', ascending=False)

,gene_id,SNPs,selected_hets,ratio
795,AGAP000951,38,6,0.157895
886,AGAP001057,19,1,0.052632
436,AGAP000530,59,2,0.033898
945,AGAP013101,52,1,0.019231
457,AGAP000552,53,1,0.018868
...,...,...,...,...
1034,AGAP028969,0,0,NaN
1055,AGAP029432,0,0,NaN
1058,AGAP029435,0,0,NaN
1061,AGAP029438,0,0,NaN


In [69]:
grouped_X.to_csv("Genes_X_spreasheet.csv")